In [1]:
# Importing dependencies
import time
import numpy as np
import cryptoaml.datareader as cdr
from collections import OrderedDict
# from cryptoaml.models import get_models 

from cryptoaml.models import XgboostAlgo 

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import f1_score
# from sklearn.model_selection import StratifiedKFold
# from evolutionary_search import EvolutionaryAlgorithmSearchCV

/home/delinvas/anaconda3/envs/btc-classifier/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
data = cdr.get_data("elliptic")
data_sets = data.train_test_split(train_size=0.7, 
                                  feat_set=["AF"], 
                                  inc_meta=False)

xgb_tune_props = {
    "method": "evolutionary_search",
    "population_size": 5,
    "generations_number": 3,
    "n_jobs": 2,
    "verbose": 0, 
    "param_grid": {
        "learning_rate": [0.05, 0.1, 0.2, 0.3], 
        "gamma": [0.0, 0.2, 0.4],
        "max_depth": [3, 6, 9],
        "min_child_weight": [1, 6, 12],
        "subsample": [0.6, 0.7, 0.8],
        "colsample_bytree": [0.5, 0.7, 0.9],
        "reg_alpha": [0, 0.5, 1],
        "reg_lambda": [0.001, 0.05, 0.1, 1]
    }
}


xgb = XgboostAlgo(tune_props=xgb_tune_props, n_jobs=2)
print(xgb.model_name_)
print(xgb.get_params())

time1 = time.time()
xgb.fit(data_sets["AF"].train_X, data_sets["AF"].train_y)
time2 = time.time()
print('{function took {:.3f} ms'.format((time2-time1)*1000.0))

test_results = xgb.evaluate(metrics=["precision", "recall", "f1", "f1_micro", "confusion"],
             X=data_sets["LF"].test_X, 
             y=data_sets["LF"].test_y)

print(test_results)
meta, results = xgb.tune_results_

print(meta)
display(results)

xg_boost
{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'gamma': None, 'gpu_id': None, 'importance_type': 'gain', 'interaction_constraints': None, 'learning_rate': None, 'max_delta_step': None, 'max_depth': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': 2, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': False, 'verbosity': None}
Types [2, 2, 1, 1, 2, 2, 1, 2] and maxint [3, 2, 2, 2, 2, 2, 2, 3] detected
--- Evolve in 11664 possible combinations ---
gen	nevals	avg     	min     	max     	std      
0  	5     	0.759872	0.745214	0.774006	0.0111676
1  	3     	0.756366	0.744674	0.774006	0.00990209
2  	4     	0.759696	0.745195	0.774273	0.0125228 
3  	4     	0.768889	0.747353	0.774273	0.0107684 
Best indivi

ValueError: unexpected '{' in field name

In [7]:
print('function took {:.3f} ms'.format((time2-time1)*1000.0))
# # Create a new instance of the Elliptic Dataset
# # and get different feature sets collection
# data = cdr.get_data("elliptic")
# data_sets = data.train_test_split(train_size=0.7, 
#                                   feat_set=["LF"], 
#                                   inc_meta=False)

function took 523134.929 ms


In [ ]:
# # Models to be tested in experiment 
# models = get_models(["xg_boost"])
# models["xg_boost"].model_name

In [ ]:
# ##########################################################################
# # Experiment 1 - Hyperparameter tuning  - TODO: Ask about param grid 
# ##########################################################################

# # Hyperparam grids for each model evaluated 
# param_grids = OrderedDict()

# # Hyperparam constants 
# TUNE_METHOD = "grid_search"
# TUNE_FOLDS  = 5

# # XGBoost param grid
# # https://xgboost.readthedocs.io/en/latest/parameter.html
# # https://xgboost.readthedocs.io/en/latest/python/python_api.html
# # https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
# scale_pos_weight = (data.label_count[0]/data.label_count[1])
# xgboost_tune = {
#     "name":   TUNE_METHOD,
#     "cv":     TUNE_FOLDS,
#     "param_grid": {
#         "learning_rate": [0.05, 0.1, 0.2, 0.3], 
#         "gamma": [0.0, 0.2, 0.4],
#         "max_depth": [3, 6, 9],
#         "min_child_weight": [1, 6, 12],
#         "subsample": [0.6, 0.7, 0.8],
#         "colsample_bytree": [0.5, 0.7, 0.9],
#         "reg_alpha": [0, 0.5, 1],
#         "reg_lambda": [0.001, 0.05, 0.1, 1],
#         "n_estimators": [100, 300, 400, 1000],
#         "scale_pos_weight": [1, scale_pos_weight]
#     }
# }

# param_grids["xg_boost"] = xgboost_tune

In [ ]:
# cv = EvolutionaryAlgorithmSearchCV(estimator=models["xg_boost"],
#                                    params=xgboost_tune["param_grid"],
#                                    scoring="f1",
#                                    cv=StratifiedKFold(n_splits=3),
#                                    verbose=1,
#                                    population_size=2,
#                                    gene_mutation_prob=0.10,
#                                    gene_crossover_prob=0.5,
#                                    tournament_size=3,
#                                    generations_number=5,
#                                    n_jobs=4)

# cv.fit(data_sets["LF"].train_X, data_sets["LF"].train_y)

In [ ]:
# test = cv.best_estimator_
# test.predict(data_sets["LF"].test_X)


In [ ]:
# ##########################################################################
# # Experiment 1 - Hyperparameter tuning  - TODO: Ask about param grid 
# ##########################################################################

# # Hyperparam grids for each model evaluated 
# param_grids = OrderedDict()

# # Hyperparam constants 
# TUNE_METHOD = "grid_search"
# TUNE_FOLDS  = 5

# # XGBoost param grid
# # https://xgboost.readthedocs.io/en/latest/parameter.html
# # https://xgboost.readthedocs.io/en/latest/python/python_api.html
# # https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
# scale_pos_weight = (data.label_count[0]/data.label_count[1])
# xgboost_tune = {
#     "name":   TUNE_METHOD,
#     "cv":     TUNE_FOLDS,
#     "param_grid": {
#         "learning_rate": [0.05, 0.1, 0.2, 0.3], 
#         "gamma": [0.0, 0.2, 0.4],
#         "max_depth": [3, 6, 9],
#         "min_child_weight": [1, 6, 12],
#         "subsample": [0.6, 0.7, 0.8],
#         "colsample_bytree": [0.5, 0.7, 0.9],
#         "reg_alpha": [0, 0.5, 1],
#         "reg_lambda": [0.001, 0.05, 0.1, 1],
#         "n_estimators": [100, 300, 400, 1000],
#         "scale_pos_weight": [scale_pos_weight]
#     }
# }

# param_grids["xg_boost"] = xgboost_tune

In [ ]:
# from sklearn.model_selection import StratifiedKFold
# from evolutionary_search import EvolutionaryAlgorithmSearchCV

# cv = EvolutionaryAlgorithmSearchCV(estimator=models["xg_boost"],
#                                    params=xgboost_tune["param_grid"],
#                                    scoring="accuracy",
#                                    cv=StratifiedKFold(n_splits=5),
#                                    verbose=1,
#                                    population_size=2,
#                                    gene_mutation_prob=0.10,
#                                    gene_crossover_prob=0.5,
#                                    tournament_size=3,
#                                    generations_number=5,
#                                    n_jobs=2)

# cv.fit(data_sets["LF"].train_X, data_sets["LF"].train_y)

In [ ]:
# ##########################################################################
# # Experiment 1 - Train/Tune/Evaluate/Save models 
# ##########################################################################
# for data_key, data_value in data_sets.items():
#     print("\nTrain/Tune/Evaluate on feature set: {}".format(data_key))

#     # Training set 
#     tmp_dataset_train_X = data_value.train_X
#     tmp_dataset_train_y = data_value.train_y
    
#     # Test set 
#     tmp_dataset_test_X = data_value.test_X
#     tmp_dataset_test_y = data_value.test_y

#     for model_key, model_value in models.items():
#         print("\nTrain/Tune: {}".format(model_key))
        
#         tmp_grid_tune = param_grids[model_key]
#         print("\nParameter Grid: {}".format(tmp_grid_tune))
#         model_value.fit(tmp_dataset_train_X, tmp_dataset_train_y, tune_props=tmp_grid_tune)
#         model_value.save("saved_models/experiment_1/{}_{}".format(model_value.model_name, data_key))
#         tmp_metrics = model_value.evaluate(metrics=["precision", "recall", "f1", "f1_micro", "confusion"], 
#                                            X_test=tmp_dataset_test_X, 
#                                            y_test=tmp_dataset_test_y)
        
#         print("\n Best Params: " + str(model_value.params))
#         print("\n" + str(tmp_metrics))
        
#         display(model_value.feature_importance.head())
#         display(model_value.feature_importance.tail())
#         print("----------------------------------------------------")
    
#     print("*********************************************************")